# Defining the Spatial Domain in CONFLUENCE
## Introduction
In this notebook, we'll explore the different methods for defining the spatial domain in CONFLUENCE. The spatial domain is crucial as it determines the area of interest for our hydrological modeling. CONFLUENCE offers several options for domain definition, each suited to different needs and data availability.

## Setup
First, let's set up our environment and import the necessary modules:

In [6]:
import sys
from pathlib import Path

# Add the parent directory to sys.path
current_dir = Path.cwd()
parent_dir = current_dir.parent
sys.path.append(str(parent_dir))

import yaml
from utils.geofabric_utils import GeofabricSubsetter, GeofabricDelineator, LumpedWatershedDelineator # type: ignore
import logging

# Load configuration
config_file = 'config_active.yaml'
config_path = parent_dir / '0_config_files' / config_file
with open(config_path, 'r') as config_file:
    config = yaml.safe_load(config_file)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Domain Definition Methods
## CONFLUENCE supports three main methods for defining the spatial domain:

1. Subsetting an existing geofabric
2. Delineating a lumped watershed
3. Delineating a new geofabric

Let's explore each of these methods.

# 1. Subsetting an Existing Geofabric

This method is useful when you have an existing geofabric and want to extract a portion of it for your study area. Confluence will subset the model domain upstream of the pour point we defined in notebook 0_initializeProject. CONFLUENCE currently supports the MERIT, TDX and NWS hydrofabrics

To use this method, set in config:

- DOMAIN_DEFINITION_METHOD: subset
- GEOFABRIC_TYPE: "Merit", "TDX" or "NWS"
- SOURCE_GEOFABRIC_BASINS_PATH: /path/to/source/geofabric_basins/geofabric_basins.shp
- SOURCE_GEOFABRIC_RIVERS_PATH: /path/to/source/geofabric_rivers/geofabric_rivers.shp
- OUTPUT_BASINS_PATH: /path/to/output/geofabric_basins/model_basins.shp
- OUTPUT_RIVERS_PATH: /path/to/output/geofabric_basins/model_basins.shp

Then call:
subset_geofabric()

In [7]:
def subset_geofabric():
    subsetter = GeofabricSubsetter(config, logger)
    subset_basins, subset_rivers = subsetter.subset_geofabric()
    
    if subset_basins is not None and subset_rivers is not None:
        logger.info("Geofabric subsetting completed successfully")
    else:
        logger.error("Geofabric subsetting failed")


## 2. Delineating a Lumped Watershed

This method creates a simple, single-unit watershed based on a pour point and DEM.

To use this method, set in config:

- DOMAIN_DEFINITION_METHOD: lumped
- DEM_PATH: path/to/your/dem

Then call: 
delineate_lumped_watershed()

In [2]:
def delineate_lumped_watershed():
    delineator = LumpedWatershedDelineator(config, logger)
    watershed_shapefile = delineator.delineate_lumped_watershed()
    
    if watershed_shapefile:
        logger.info(f"Lumped watershed delineation completed. Shapefile: {watershed_shapefile}")
    else:
        logger.error("Lumped watershed delineation failed")

## 3. Delineating a New Geofabric

This method creates a new, detailed geofabric for your study area using a DEM and pour point. This method uses the TauDEM (Terrain Analysis Using Digital Elevation Models) tools for watershed delineation which are available at https://github.com/dtarb/TauDEM. To use this functionality the TauDEM binaries need to be compiled and made available in the system PATH

To use this method, set in config:

- DOMAIN_DEFINITION_METHOD: delineate
- DEM_PATH: path/to/your/dem
- STREAM_THRESHOLD: Threshold flow accumulation for stream vectorisation
- TAUDEM_DIR: path/to/TauDEM/binaries

Then call: delineate_geofabric()

In [8]:
def delineate_geofabric():
    delineator = GeofabricDelineator(config, logger)
    river_network_path, river_basins_path = delineator.delineate_geofabric()
    
    if river_network_path and river_basins_path:
        logger.info(f"Geofabric delineation completed.")
        logger.info(f"River network: {river_network_path}")
        logger.info(f"River basins: {river_basins_path}")
    else:
        logger.error("Geofabric delineation failed")
        

## Choosing the Right Method
The choice of domain definition method depends on your specific needs:

- Use subset_geofabric if you have an existing geofabric and want to focus on a specific area within it.
- Use delineate_lumped_watershed for simple, single-unit watershed studies or as a quick approximation.
- Use delineate_geofabric when you need a detailed, custom geofabric for your study area.

Remember to set the appropriate method in your configuration file before running the domain definition.

## Running the Domain Definition

Here's how you can run the domain definition based on the configuration:

In [5]:
def define_domain():
    domain_method = config.get('DOMAIN_DEFINITION_METHOD')
    
    if domain_method == 'subset':
        subset_geofabric()
    elif domain_method == 'lumped':
        delineate_lumped_watershed()
    elif domain_method == 'delineate':
        delineate_geofabric()
    else:
        logger.error(f"Unknown domain definition method: {domain_method}")

# Run the domain definition
define_domain()

2024-09-14 16:21:46,530 - ERROR - Unknown domain definition method: delineate


# Conclusion

We've explored the three methods for defining the spatial domain in CONFLUENCE: subsetting an existing geofabric, delineating a lumped watershed, and delineating a new geofabric. Each method has its advantages and is suited to different scenarios. By choosing the appropriate method and configuring it correctly, you can define the spatial domain that best fits your hydrological modeling needs.

In the next notebook, we'll look at how to process finalise the process of preparing your spatial domain representation, including optional domain discretisation by dominant geospatial attributes.